In [2]:
%display typeset

# Mapas de Dimensão finita
Também conhecidos como equações de diferença são sistemas dinâmicos sobre tempo discreto. Neste Notebook iremos explorar alguns resultados básicos sobre mapas discretos e mapas clássicos em uma e duas dimensões.

Vamos começar com um exemplo simples:

$$U_{n+1}=a sin(U_n), \,\,U_0=U$$

A partir da expressão acima vemos que Mapas são funções de $\mathbb{R}^p \rightarrow \mathbb{R}^p$ que produzem sequências de números da forma $\{U_n\}_{n=0}^{\infty}$. No caso de mapas unidimensionais, $p=1$.

In [2]:
@interact
def oneDmap(a=slider(0.5,3,.1,1.1, label='$a$'), u0=slider(0,1,.1,.1, label='$U_0$'), n=slider(10,300,1,100, label='Iterações:')):
    pts = [(0,a*sin(u0))]
    for i in range(1,n+1):
        nu = a*sin(pts[-1][1])
        pts.append((i,nu))
    p = points(pts,axes_labels=['$n$', '$U_n$'])
    show("Uf= ",pts[-1][1])
    show(p)
        

Interactive function <function oneDmap at 0x7f64d74ec040> with 3 widgets
  a: TransformFloatSlider(value=1.1, …

### Mapa quadrático
Como um segundo exemplo, vamos considerar o mapa quadrático:
$$U_{n+1}=a U_n (1-U_n), \,\,\, U_0=U$$

In [3]:
qmap = lambda u,a: a*u*(1-u)
@interact
def quad_map(a=slider(0.5,4.5,.1,1.1, label='$a$'), u0=slider(0,1,.1,.1, label='$U_0$'), n=slider(10,300,1,100, label='Iterações:')):
    pts = [(u0, qmap(u0,a))]
    for i in range(1,n+1):
        nu = qmap(pts[-1][1],a)
        pts.append((i,nu))
    p = points(pts,axes_labels=['$n$', '$U_n$'])
    show("Uf= ",pts[-1][1])
    show(p)

Interactive function <function quad_map at 0x7f64d7d9aa60> with 3 widgets
  a: TransformFloatSlider(value=1.1,…

## Diagramas Cobweb (teia de aranha)

Também conhecidos como diagramas de Verhulst, diagramas cobweb nos permitem explorar a dinâmica de mapas discretos. vejamos o exemplo abaixo com um mapa quadrático também conhecido como mapa logístico. 

$$U_{n+1} = a U_n (1-U_n)$$

Estes diagramas podem ser construídos utilizado-se os seguintes passos:
1. Encontre o ponto $(U_0, U_1)$;
1. Trace uma linha horizontal deste ponto até encontrar a diagonal no ponto $(U_1,U_1)$;
1. Trace uma linha vertical deste último ponto na diagonal até encontrar a função no ponto $(U_1, U_2)$;
1. Repita os passos 2 e 3 incrementando os indices.

No diagrama de Teia de aranha, um *ponto fixo* aparece como uma **espiral convergente**, Um *ponto instável* como uma **espiral divergente**, Uma *órbita de período 2* aparece como um **retângulo**, *períodos maiores* aparecem como **múltiplos retângulos**. *Atratores caóticos* aparecem como **inúmeros retângulos**. 

In [4]:
def cobweb(a_function, start, mask = 0, iterations = 20, xmin = 0, xmax = 1):
    '''
    Returns a graphics object of a plot of the function and a cobweb trajectory starting from the value start.

    INPUT:
        a_function: a function of one variable
        start: the starting value of the iteration
        mask: (optional) the number of initial iterates to ignore
        iterations: (optional) the number of iterations to draw, following the masked iterations
        xmin: (optional) the lower end of the plotted interval
        xmax: (optional) the upper end of the plotted interval
    
    EXAMPLES:
        sage: f = lambda x: 3.9*x*(1-x)
        sage: show(cobweb(f,.01,iterations=200), xmin = 0, xmax = 1, ymin=0)
    
    '''
    basic_plot = plot(a_function, xmin = xmin, xmax = xmax, axes_labels=['$U_n$', '$U_{n+1}$'])
    iv = point((start,a_function(start)),color='green', size=30)
    id_plot = plot(lambda x: x, xmin = xmin, xmax = xmax)
    iter_list = []
    current = start
    for i in range(mask):
        current = a_function(current)
    for i in range(iterations):
        iter_list.append([current,a_function(current)])
        current = a_function(current)
        iter_list.append([current,current])
    series = list_plot([[i,pt[0]] for i,pt in enumerate(iter_list)], color='red', legend_label='$U_n$')
    cobweb = line(iter_list, rgbcolor = (1,0,0))
    ga = graphics_array([
        [basic_plot +iv + id_plot + cobweb],
        [series]
    ],)
    ga.show(figsize=[8,10])#basic_plot +iv + id_plot + cobweb
var('x')
@interact
def cobwebber(f_text = input_box(default = "a*x*(1-x)",label = "mapa", type=str),
              a = slider(1,4,0.1,3.8, label='$a$'),
              start_val = slider(0,1,.01,.5,label = 'valor inicial'), 
              its = slider([2^i for i in range(0,12)],default = 16, label="iterações")):
    def f(x):
        return eval(f_text.replace('a',str(a)))
    show(cobweb(f, start_val, iterations = its))

Interactive function <function cobwebber at 0x7f64cdb48ee0> with 4 widgets
  f_text: TransformText(value='a*x*…

### Diagrama orbital
Vimos que o número de orbitas distintas no modelo logístico varia com o parâmetro $a$. Podemos construir outro diagrama interessante representando o número de órbitas ao longo de um intervalo do parâmetro.

In [5]:
import numpy as np
logmap = lambda a,x: a*x*(1-x)
@interact
def orb_diagram(mask=50, start=0.5, a_min=slider(2.9,4.0,.01,3.4), a_max=slider(3.5,4.0,.01,4),
                ymax = slider(0,1,0.01,1),
               its = slider([2^i for i in range(0,12)],default = 16, label="iterações")):
    func = logmap
    states = []
    for a in np.linspace(a_min,a_max,150, dtype=float):
        iter_list = [] 
        current = start
        for i in range(mask):
            current = func(a,current)
        for i in range(its):
            iter_list.append([a,func(a, current)])
            current = func(a,current)
#             iter_list.append([a,current])
        states.extend(iter_list)
    p = list_plot(states)
    p.show(figsize=[8,8],ymax=ymax)


orb_diagram(logmap)

Interactive function <function orb_diagram at 0x7f64cdb3c160> with 6 widgets
  mask: IntSlider(value=50, descr…

TypeError: unsupported operand type(s) for *: 'TransformFloatSlider' and 'float'

In [6]:
a_min=slider(3.5,4,.01,3.4)
a_min.value

3.5

## Mapa de Hénon, um mapa bi-dimensional
Stuart e Humphries apresentam no capítulo 1, equação 1.1.3 o mapa de Henón:


\begin{align}
X_{n+1} &= a -bY_n -X^2_n\\
Y_{n+1} &= X_n
\end{align}


In [7]:
@interact
def Henon_map(x0=slider(-0.4,.4,.1,.1, label='$X_0$'), 
              y0=slider(-.4,.4,.1,.1, label='$Y_0$'),
              a=slider(0,1,.1,0,label='$a$'),
              b=slider(0,1,.1,1,label='$b$'),
              n=slider(10,1000,50,500, label='Iterações:')):
    pts = [(a-b*y0-x0**2,x0)]
    for i in range(1,n+1):
        xi = a-b*pts[-1][1]-pts[-1][0]**2
        yi = pts[-1][0]
        pts.append((xi,yi))
    p = points(pts, axes_labels=['$X_n$', '$Y_n$'])
    xp = list_plot([[i,pt[0]] for i,pt in enumerate(pts)], color='red', legend_label='$X_n$')
    yp = list_plot([[i,pt[1]] for i,pt in enumerate(pts)], color='green', legend_label='$Y_n$', alpha=.6)
    ga = graphics_array((p,xp+yp))
    ga.show(figsize=[10,5])

Interactive function <function Henon_map at 0x7f64cd9c9160> with 5 widgets
  x0: TransformFloatSlider(value=0.…

Entretando o Mapa de Hénon [mais conhecido](https://en.wikipedia.org/wiki/H%C3%A9non_map) tem uma formulação ligeiramente diferente:

\begin{align}
X_{n+1} &= 1 -a X^2_n + Y_n \\
Y_{n+1} &= b X_n
\end{align}

É importante mencionar que o sistema acima é a forma como o mapa aparece no [artigo original de Hénon](https://venturi.soe.ucsc.edu/sites/default/files/Henon1976.pdf) de 1976.

In [8]:
@interact
def Henon_map_2(x0=slider(-0.4,.4,.1,0, label='$X_0$'), 
              y0=slider(-.4,.4,.1,0, label='$Y_0$'),
              a=slider(0,2,.1,1.2,label='$a$'),
              b=slider(0,1,.1,0.3,label='$b$'),
              n=slider(10,1000,50,500, label='Iterações:')):
    pts = [(a-b*y0-x0**2,x0)]
    for i in range(1,n+1):
        xi = 1-a* pts[-1][0]**2 + pts[-1][1]
        yi = b*pts[-1][0]
        pts.append((xi,yi))
    p = points(pts, axes_labels=['$X_n$', '$Y_n$'])
    xp = list_plot([[i,pt[0]] for i,pt in enumerate(pts)], color='red', legend_label='$X_n$')
    yp = list_plot([[i,pt[1]] for i,pt in enumerate(pts)], color='green', legend_label='$Y_n$', alpha=.3)
    ga = graphics_array((p,xp+yp))
    ga.show(figsize=[10,5])

Interactive function <function Henon_map_2 at 0x7f64cd3f9160> with 5 widgets
  x0: TransformFloatSlider(value=…

## O Semigrupo $S^n$.
Vamos introduzir o subgrupo $S^n$ como uma notação para a realização de n passos(iterações) do mapa.

Dado um sistema dinâmico sobre um conjunto $E$. Podemos definir o semigrupo de evolução $S^n:E \rightarrow E$ como sendo o operador tal que $U_n=S^n U_0$. Este operador tem as seguintes propriedades:
1. $U_{n+m}=S^nU_m=S^mU_n=S^{n+m}U_0$, para todo $n,m \geq 0$;
1. $S^0= I $, a identidade.

o semigrupo $S^n$ é comumente chamado de *mapa de evolução*.

## Conjuntos Limite (Limit sets)
As seguintes perguntas são relevantes para sistemas dinâmicos de qualquer natureza: 
 - Quais os seus equilíbrios? 
 Pontos de equilíbrio são estados de onde o sistema não sairá a menos que perturbado. No caso dos mapas discretos estes equilíbrios são chamados de conjuntos limites, pois podem ser definidos por um conjunto de pontos ao invés de um único.
 - Quais condições iniciais levam a que conjuntos limites?
 ou seja, que conjuntos de pontos constituem sua **bacia de atração**?
 
Considere o seguinte mapa: $$U_{n+1}=U_n^2,\,\,U_0=U$$

Este é um sistema dinâmico em $\mathbb{R}^+$. É óbvio que se $U<1$, $U_n \rightarrow 0$ quando $n\rightarrow \infty$. Logo, $0$ é um ponto fixo. O conjunto $[0,1)$ é a bacia de atração de 0. Se $U=1$ então $U_n=1, \forall n$. Logo $1$ também é um ponto fixo. Por fim, se $U>1$ então $U_n \rightarrow \infty$ quando $n\rightarrow \infty$. Portanto $\infty$ é um equilíbrio do sistema, e o conjunto $(1,\infty]$ é sua bacia de atração.

Encontrar os equilíbrios de uma Mapa nem sempre é tão simples. Consider este outro mapa: 
$$U_{n+1}=a U -U^3, \, U_0=U$$


In [10]:
@interact
def cobwebber(f_text = input_box(default = "a*x - x**3",label = "mapa", type=str),
              a = slider(-1,4,0.1,3, label='$a$'),
              start_val = slider(0,2,.01,n(sqrt(3)),label = 'valor inicial'), 
              its = slider([2^i for i in range(0,12)],default = 16, label="iterações")):
    def f(x):
        return eval(f_text.replace('a',str(a)))
    show(cobweb(f, start_val, iterations = its,xmin=-2,xmax=2))

Interactive function <function cobwebber at 0x7f64c7064c10> with 4 widgets
  f_text: TransformText(value='a*x …

## Estabilidade
Pontos fixos por definição são valores de $U$ para os quais $U_{n+1}=U_n$. Logo quando existirem sempre pertencerão à diagonal do gráfico de cobweb.

Para uma definição mais formal, considere o seguinte mapa linear:
$$U_{n+1}=AU_n.$$

$A$ tem autovalores $\{\lambda_i\}_{i=1}^l$, onde $l\leq p$. Logo,
1. A origem é assintoticamente estável se e somente se $|\lambda_i|<1$. para todos os $i$.
2. Se $|\lambda_i|\leq1$ para todo $i$ e os autovalores iguais a um são "não-defeituosos", então a origem é estável.

Para mapas não lineares pode-se aplicar técnicas de linearização local nos pontos fixos, e aplicar os critérios acima. 

### Analisando o Mapa Logístico
Vamos revisitar o Mapa logístico e analisar seus pontos fixos. Vamos denotar por $x^*$ os pontos fixos.
$$x_{n+1}=rx_n(1-x_n)$$

Os pontos fixos satisfazem $x^* = f(x^*)=rx^*(1-x^*)$. Logo $x^*=0$ ou $x^*=1-\frac{1}{r}$ 

In [3]:
var('r x')
solve(x==r*x*(1-x),x)

[x == (r - 1)/r, x == 0]

A estabilidade dos pontos fixos depende da derivada de $f(x^*)$, $f'(x^*) = r - 2rx^*$. NO caso de $x^*=0$, $f'(0)=r$ este ponto fixo é estável se $r<1$ e instável se $r>1$. Para o segundo ponto fixo, $f'(x^*)=r-2r(1-1/r)=2-r$. Logo o segundo ponto fixo é estável quando $1<r<3$ e instável quando $r>3$.